# VALUE AT RISK

Abstract Focusing on the creation of portfolios for investment, this chapter aims to understand the risks of the portfolio through methods such as the Value at Risk (VaR) to determine the possible loss or gain of a portfolio. This chapter is based on an investor view and the process for executing decisions that create profitable portfolios in the short and long run.

Keywords Risk · Portfolios · VaR · Backtesting

The concept of Value at Risk (VaR) is one of the most interesting in finance because it analyzes the maximum loss that a portfolio may have (Damodaran 2018). This is another measure of risk that deserves to be separated from portfolio and risk because of the difference that it has with the ratios (Sharpe, Traynor, Information and Jensen) in the previous chapter. To summarize the VaR, it gives the worst loss on a certain time horizon based on the confidence level assigned to the model.

## Historical VAR(95)

Since the VaR is based on the confidence level, it may have different results based on a 65%, 90%, 95% or any other confidence interval. The following example is Historical VaR(95), meaning that the confidence interval will be at a 95%.


In [27]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
from scipy.stats import norm

%matplotlib inline


### • Choose the portfolio

In [28]:
start=datetime.datetime(2018,1,2)
end=datetime.datetime(2019,4,1)

tickers=['AAPL','WMT','TM','KO','BA']

stocks=pd.DataFrame()
for x in tickers:
    stocks[x]=yf.download(tickers=x,start=start,end=end)['Close']

stocks.tail()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,47.185001,98.169998,120.489998,46.029999,370.459991
2019-03-26,46.697498,98.320000,121.769997,46.639999,370.380005
2019-03-27,47.117500,97.209999,121.029999,46.610001,374.209991
2019-03-28,47.180000,97.129997,119.070000,46.580002,374.440002
2019-03-29,47.487499,97.529999,118.019997,46.860001,381.420013


### • Calculate the returns

In [29]:
stocks_return=np.log((stocks/stocks.shift(1)))
stocks_return.tail()

,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,-0.012165,-0.001120,0.002243,0.002175,0.022632
2019-03-26,-0.010385,0.001527,0.010567,0.013165,-0.000216
2019-03-27,0.008954,-0.011354,-0.006096,-0.000643,0.010288
2019-03-28,0.001326,-0.000823,-0.016327,-0.000644,0.000614
2019-03-29,0.006496,0.004110,-0.008857,0.005993,0.018470


### • Assign random portfolio weights that sum to one (1)

In [30]:
portfolio_weights = np.array(np.random.random(5))
portfolio_weights
portfolio_weights=portfolio_weights/np.sum(portfolio_weights)
portfolio_weights


array([0.12354394, 0.31764259, 0.16961075, 0.08193349, 0.30726923])

This step is interesting because, in the Portfolio and Risk chapter, the purpose was to assign the same return to each of the stocks. In this case the np.random.random creates weights for the five (5) stocks but it often gives a number less or higher than 100%. Therefore it has to be balanced by dividing the weights in the sum to obtain a portfolio that sums 100%.

### • Multiply the portfolio with the stocks

In [31]:
weighted_returns_portfolio = stocks_return.mul(portfolio_weights,axis=1)
weighted_returns_portfolio.tail()

,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,-0.001503,-0.000356,0.000380,0.000178,0.006954
2019-03-26,-0.001283,0.000485,0.001792,0.001079,-0.000066
2019-03-27,0.001106,-0.003606,-0.001034,-0.000053,0.003161
2019-03-28,0.000164,-0.000262,-0.002769,-0.000053,0.000189
2019-03-29,0.000803,0.001305,-0.001502,0.000491,0.005675


###• Convert returns to percentages and drop the missing values

In [32]:
stocks_return['Portfolio'] = weighted_returns_portfolio.sum(axis=1).dropna()
stocks_return['Portfolio'] = stocks_return['Portfolio'] * 100
stocks_return.tail()

,AAPL,WMT,TM,KO,BA,Portfolio
Date,,,,,,
2019-03-25,-0.012165,-0.001120,0.002243,0.002175,0.022632,0.565411
2019-03-26,-0.010385,0.001527,0.010567,0.013165,-0.000216,0.200656
2019-03-27,0.008954,-0.011354,-0.006096,-0.000643,0.010288,-0.042580
2019-03-28,0.001326,-0.000823,-0.016327,-0.000644,0.000614,-0.273091
2019-03-29,0.006496,0.004110,-0.008857,0.005993,0.018470,0.677188


### • Calculate the VaR95

In [33]:
var95=np.percentile(stocks_return['Portfolio'], 5)
var95

-2.0418252925742353

Based on the historical returns of the portfolio at a 95% confidence interval, the worst loss is a var95 loss, therefore the result is negative.

## Historical VAR(99)

For computing the Historical VaR at a 99% confidence level the only change that has to be done is in the last part of the script, changing the np.percentile to 1, which means the 1%.

In [34]:
var99=np.percentile(stocks_return['Portfolio'], 1)
var99

-3.3799245300431653

At a 99% confidence level the worst loss is var99 with the portfolio. Clearly the VaR is higher given that the confidence level is lower. This is rational and therefore it helps understand the process by which the VaR works, given that a higher confidence level will give a higher percentage of loss and a lower confidence level will give a lower percentage of loss.

## VAR FOR THE NEXT 10 DAYS

One of the most important aspects of calculating a VAR is to calculate the effect on the investment in terms of money. As far, the VAR model has centered on the percentage loss, but for the next example the process is to analyze the VAR if USD 1 million is invested. For example, the same data set will be used.

### • Choose the portfolio

In [35]:
start=datetime.datetime(2018,1,2)
end=datetime.datetime(2019,4,1)

tickers=['AAPL','WMT','TM','KO','BA']

stocks=pd.DataFrame()

for x in tickers:
    stocks[x]=yf.download(tickers=x,start=start,end=end)['Close']

stocks.tail()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,47.185001,98.169998,120.489998,46.029999,370.459991
2019-03-26,46.697498,98.320000,121.769997,46.639999,370.380005
2019-03-27,47.117500,97.209999,121.029999,46.610001,374.209991
2019-03-28,47.180000,97.129997,119.070000,46.580002,374.440002
2019-03-29,47.487499,97.529999,118.019997,46.860001,381.420013


### • Calculate the returns

In [36]:
stocks_return=np.log((stocks/stocks.shift(1)))
stocks_return.tail()

,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,-0.012165,-0.001120,0.002243,0.002175,0.022632
2019-03-26,-0.010385,0.001527,0.010567,0.013165,-0.000216
2019-03-27,0.008954,-0.011354,-0.006096,-0.000643,0.010288
2019-03-28,0.001326,-0.000823,-0.016327,-0.000644,0.000614
2019-03-29,0.006496,0.004110,-0.008857,0.005993,0.018470


### • Assign random portfolio weights that sum to one (1)

In [37]:
portfolio_weights = np.array(np.random.random(5))
portfolio_weights
portfolio_weights=portfolio_weights/np.sum(portfolio_weights)
portfolio_weights

array([0.2120737 , 0.13077686, 0.09992969, 0.28968075, 0.267539  ])

### • Multiply the portfolio with the stocks

In [38]:
weighted_returns_portfolio = stocks_return.mul(portfolio_weights,axis=1)

weighted_returns_portfolio.tail()

,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,-0.002580,-0.000146,0.000224,0.000630,0.006055
2019-03-26,-0.002202,0.000200,0.001056,0.003814,-0.000058
2019-03-27,0.001899,-0.001485,-0.000609,-0.000186,0.002752
2019-03-28,0.000281,-0.000108,-0.001632,-0.000187,0.000164
2019-03-29,0.001378,0.000537,-0.000885,0.001736,0.004941


### • Calculate the returns based on the weights

In [39]:
stocks_return['Portfolio']=weighted_returns_portfolio.sum(axis=1).dropna()
stocks_return.tail()

,AAPL,WMT,TM,KO,BA,Portfolio
Date,,,,,,
2019-03-25,-0.012165,-0.001120,0.002243,0.002175,0.022632,0.004183
2019-03-26,-0.010385,0.001527,0.010567,0.013165,-0.000216,0.002809
2019-03-27,0.008954,-0.011354,-0.006096,-0.000643,0.010288,0.002371
2019-03-28,0.001326,-0.000823,-0.016327,-0.000644,0.000614,-0.001480
2019-03-29,0.006496,0.004110,-0.008857,0.005993,0.018470,0.007707


### • Determine the average (mu) of the returns

In [40]:
mu=stocks_return['Portfolio'].mean()
mu

0.0002765023659876238

### • Determine the standard deviation (sigma) of the returns

In [41]:
sigma=stocks_return['Portfolio'].std()
sigma

0.010947913019399394

### • Assign a confidence level to the VaR (99% for this example)

In [42]:
confidence=0.99
confidence

0.99

### • Calculate the alpha

In [43]:
alpha=norm.ppf(1-confidence)
alpha

-2.3263478740408408

For this example, the norm.ppf is being used, the reason for this is that it determines the probability density function of one (1) minus the confidence interval. This is useful because it determines the probability of the VAR. It is a similar process to the np.percentile.

### • Create a position


In [44]:
position=1e6
position

1000000.0

The position is the investment on the portfolio. Since the portfolio was created, in this case the investment is USD 1 million. The interesting aspect of using 1e6 for a million is to include a complex number structure that is easier to write. The other choice would have been to write the 1,000,000.

### • Calculate the VAR

Equation 1: Value at Risk - position

    VaR = position * (µ − σ * α)

µ = mean of the returns of the portfolio

σ = standard deviation of the returns of the portfolio

α = Probability density function of the 1%




In [45]:
var=position*(mu-sigma*alpha)
var

25745.156543851444

If the investment in the portfolio was of USD 1,000,000, the worst loss at a 99% confidence interval can be of USD 27,088.75. The next step is to obtain the VAR for the next 10 days, trying to identify what will be the loss of the portfolio.

### • Create a variable for 10 days

In [46]:
days=10
days

10

### • Determine the worst loss for the next 10 days

    VaR for 10 days = position ∗ (µ ∗ days − σ ∗ α ∗ sqrt(days))


In [47]:
var_10_days=position *(mu*days-sigma*alpha*np.sqrt(days))
var_10_days

83303.97980108907

The worst loss for the next 10 days based on the portfolio that depends on the stocks that have been chosen and the weights of the stocks, could be of USD 88,644.95 or approximately 88.64% of the total investment. Consider that this effect is at a 99% confidence interval. If the example had been done with a 95% of confidence interval, the result would have been as follows:

### • Assign a confidence interval of 95%

In [48]:
confidence=0.95
confidence

0.95

### • Obtain the alpha

In [49]:
alpha=norm.ppf(1-confidence)
alpha

-1.6448536269514722

### • Determine the worst loss for the next 10 days

In [50]:
var_10_days=position *(mu*days-sigma*alpha*np.sqrt(days))
var_10_days

59710.416736300445

The result is a worst loss much smaller than the one determined at a 99% confidence level. In this example the loss is approximately 63.95% of the total investment. The result also varies if the days are reduced, for example to 5 days.

### • Worst loss for the next 5 days at a 95% confidence level

In [51]:
days_2=5
var_5_days=position *(mu*days_2-sigma*alpha*np.sqrt(days))
var_5_days

58327.90490636233